# Team EGE - BLG453E Term Project
Egehan Orta - 150160124

Yavuz Ege Okumuş - 150160118
## Dice Game

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from pyautogui import screenshot, press
import time

### PART 1

In [ ]:
keys = ['A', 'S', 'D']
time.sleep(2)
while True:
    img = np.array(screenshot())
    gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    gray[gray<200] = 0
    gray[gray>=200] = 255
    edges = cv.Canny(gray, 30, 50)  
    contours = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    dices = []
    for cnt in contours[0]:
        (x,y,w,h) = cv.boundingRect(cnt)
        ROI = edges[y:y+h,x:x+w]
        ROI = cv.blur(ROI, (3,3))
        ROI[ROI<75] = 0
        ROI[ROI>=75] = 255
        dices.append((ROI,x+w))
    dices.sort(key=lambda x: x[1])
    circles = []
    for dice,_ in dices:
        circle = cv.HoughCircles(dice,cv.HOUGH_GRADIENT,0.5, 20, param1 = 50, 
                   param2 = 30, minRadius = 0, maxRadius = 0)
        circles.append(len(circle[0]))
    press(keys[np.argmax(circles)])
    time.sleep(1)

### PART 2

In [ ]:
def dice_detector(img):
    circles = set()
    thresh = img.copy()
    conts = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)   
    for i in conts[0]:
        if  cv.contourArea(i) > 65000:
            empty = np.zeros_like(img)
            cv.fillPoly(empty, [i], (255,255,255))
            result = cv.bitwise_and(img, empty)
            white = cv.drawContours(result.copy(), [i], -1, (255), -1)         
            corners = cv.goodFeaturesToTrack(white, 4, 0.3, 10)
            pts = np.float32([[0,0], [300,0], [0,300], [300, 300]])
            corners=corners.reshape(-1,2)
            
            if corners.size == 8:
                dist = 0
                cross = None
                
                for c in range(1,4):
                    if np.sum((corners[0]-corners[c])**2)>dist:
                        dist = np.sum((corners[0]-corners[c])**2)
                        cross = c
                    
                if cross == None:
                    continue
                if cross == 1:
                    corners = corners[[0,2,3,1]]
                elif cross == 2:
                    corners = corners[[0,1,3,2]]
                    
                M = cv.getPerspectiveTransform(corners,pts)
                dst = cv.warpPerspective(thresh,M,(300,300))
                
                edges = cv.Canny(dst, 30, 50)
                
                ROI = cv.blur(edges, (3,3))
                ret, ROI = cv.threshold(ROI,75,255,cv.THRESH_BINARY)
                circle = cv.HoughCircles(ROI,cv.HOUGH_GRADIENT,0.5, 20, param1 = 50, 
                                         param2 = 30, minRadius = 0, maxRadius = 0)
                
                circles.add(len(circle[0]))
    if len(circles) == 1:
        return circles.pop()
    else:
        return False

In [ ]:
begin = True
time.sleep(2)
while True:
    if begin:
        numbers = [0,0,0]
    im = screenshot()
    gray = cv.cvtColor(np.uint8(im), cv.COLOR_RGB2GRAY)
    ret,thresh = cv.threshold(gray,200,255,cv.THRESH_BINARY)
    left = thresh[:,:670]
    mid = thresh[:,670:1260]
    right = thresh[:,1260:]
    
    if numbers[0] == 0:
        numbers[0] = dice_detector(left)
    if numbers[1] == 0:
        numbers[1] = dice_detector(mid)
    if numbers[2] == 0:
        numbers[2] = dice_detector(right)
    if not all(numbers):
        begin = False
    else:
        press('ASD'[np.argmax(numbers)])
        begin = True
    time.sleep(1)